In [1]:
import json_lines

In [2]:
import json

In [3]:
import gensim

In [4]:
import re

In [5]:
import numpy as np

In [6]:
from string import punctuation
translation = str.maketrans("","", punctuation)

In [7]:
data = []
with open('items.jl', 'r') as f:
    for item in json_lines.reader(f):
        data.append(item)

In [8]:
topics = {}
for i in data:
    for t in i['topic']:
        if t not in topics:
            topics[t] = 1
        else:
            topics[t] += 1

In [9]:
for i in sorted(topics, key=topics.get, reverse=True)[:50]:
    print(i+': ', str(topics[i]))

Career Advice:  77
Exams and Tests:  75
India:  60
College Advice:  52
Life Advice:  44
Dating and Relationships:  43
Jobs and Careers:  43
Medicine and Healthcare:  38
Politics of the United States of America:  38
Physics:  35
Computer Programming:  33
Religion:  33
Education:  32
Life and Living:  31
Dating Advice:  29
Personal Question:  28
Engineering:  26
The United States of America:  26
Human Behavior:  25
Interpersonal Interaction:  25
Food:  23
Philosophy of Everyday Life:  22
Mathematics:  22
Chemistry:  21
Psychology of Everyday Life:  20
Relationship Advice:  20
Pakistan:  20
Cars and Automobiles:  19
Quora:  19
Donald Trump (politician, business person):  19
International Relations:  18
English (language):  18
Business:  18
Colleges and Universities:  17
Books:  17
Smartphones:  17
College and University Admissions:  17
Studying:  17
Countries of the World:  16
Health:  16
Banking:  16
Web Development:  16
Learning to Program:  15
Politics:  15
Programming Languages:  15
S

In [9]:
noaq = re.compile('\?$')
noacom = re.compile('\.{3}$')

In [10]:
data = data[1:]

In [11]:
def cleanup(data):
    # data = list(set(data))
    for i in data:
        i['q'] = i['q'][0]
        arr = []
        for a in i['a']:
            if not noaq.match(a) and not noacom.match(a):
                arr.append(a)
        i['a'] = ' '.join(arr)
    return data

In [12]:
data = cleanup(data)

In [10]:
clean = open('clean.json', 'w')

In [12]:
json.dumps(data, clean, indent=4)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [13]:
m = 'GoogleNews-vectors-negative300.bin.gz'
if m.endswith('.vec.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=False, limit=500000)
elif m.endswith('.bin.gz'):
    model = gensim.models.KeyedVectors.load_word2vec_format(m, binary=True, limit=500000)
else:
    model = gensim.models.KeyedVectors.load_word2vec_format(m, limit=500000)

In [14]:
model.init_sims(replace=True)

In [15]:
model.most_similar('dog')

[('dogs', 0.8680489659309387),
 ('puppy', 0.8106428980827332),
 ('pit_bull', 0.7803961038589478),
 ('pooch', 0.7627376914024353),
 ('cat', 0.760945737361908),
 ('golden_retriever', 0.7500901222229004),
 ('German_shepherd', 0.7465174198150635),
 ('Rottweiler', 0.7437615394592285),
 ('beagle', 0.7418621778488159),
 ('pup', 0.7406911253929138)]

In [16]:
seqs = []
unknown = {}
vw = []

In [17]:
def prepare_for_conv(data, seqs=[]):
    for i in data:
        d = {}
        d['q'] = []
        d['a'] = []
        for word in i['q'].translate(translation).split():
            try:
                d['q'].append(model[word])
                vw.append((model[word], word))
            except KeyError:
                if word not in unknown:
                    vw.append((v, word))
                    v = np.random.rand(300)
                    d['q'].append(v)
                    vw.append((v, word))
                    unknown[word] = v
        for word in i['a'].translate(translation).split():
            try:
                d['a'].append(model[word])
                vw.append((model[word], word))
            except KeyError:
                v = np.random.rand(300)
                d['a'].append(v)
                vw.append((v, word))
                unknown[word] = v
        seqs.append(d)
    return seqs

In [18]:
seqs = prepare_for_conv(data, seqs=seqs)

In [19]:
vw[0]

(array([-0.00868645,  0.05772586,  0.05735818,  0.01902747, -0.09375856,
         0.02996597,  0.05772586, -0.08052205,  0.06618251,  0.07096236,
         0.04485704, -0.11251027, -0.05257833,  0.02224468, -0.09633233,
         0.09339087,  0.01893555,  0.01553451,  0.0255538 , -0.026473  ,
         0.00772129,  0.05000456,  0.09339087, -0.08015437, -0.00058886,
         0.03070133, -0.02178508, -0.03860646,  0.04301863,  0.01553451,
        -0.02150932, -0.04007719, -0.00657229, -0.00661825,  0.01296074,
         0.0332751 , -0.01856787,  0.06985932,  0.02684068,  0.1213346 ,
        -0.03934183,  0.04191559,  0.01682139, -0.0465116 , -0.04283479,
         0.01535067, -0.04375399, -0.02757605,  0.02261236,  0.08456654,
        -0.0797867 ,  0.10736274, -0.02068204, -0.0683886 ,  0.09192015,
         0.03989335, -0.01204154, -0.09192015,  0.0948616 ,  0.01525875,
        -0.00131561, -0.00356191, -0.07280076,  0.00597481, -0.02757605,
        -0.05368137,  0.03437814,  0.00799705,  0.0

In [23]:
import tensorflow as ff
import data_utils as du
import seq2seq_model as seqseq

In [24]:
from sklearn.model_selection import train_test_split as skspl

In [25]:
qs = [i['q'] for i in data]
ans = [i['a'] for i in data]

In [28]:
ans[6]

"Depends of what type of person you are and of what you want. Mathematics is perfection, is fiction, is pure logic. You can have two points of view on mathematics: It is a world that  you build  by creating your functions, setting your rules, seing what happens then. Or it is a world that starts to exists once you have settled the undemonstrable basic rules and that  you discover. Physics is also pretty much perfect in its theories but is way more applicative. You observe and measure. From that you build models that have the goal to fit the most yours observations. And then, you do your best to  prove yourself wrong.  If you fail to prove yourself wrong, then maybe your theory/model is close to reality. Often researchers discover/measure new things that will disprove models and in these cases new models are created to fit the “measured” reality. And the “ultimate goal” is to find a model that answers everything (energetic, matter, laws of physics, aso..) I would say that both are reall

In [30]:
X_train, X_test, y_train, y_test = skspl(qs, ans, test_size=0.15)

In [31]:
with open('train.en', 'w') as en:
    for i in X_train:
        en.write(i+'\n')

In [32]:
with open('train.fr', 'w') as en:
    for i in y_train:
        en.write(i+'\n')

In [33]:
with open('test.en', 'w') as en:
    for i in X_test:
        en.write(i+'\n')

In [34]:
with open('test.fr', 'w') as en:
    for i in y_test:
        en.write(i+'\n')

In [ ]:
/home/algernon/quora_links